# Прогноз результатов матчей на основе TrueSkill

Загружаем нужные библиотеки.

In [1]:
import pandas as pd
import trueskill

from math import sqrt, log, log2
from trueskill import BETA
from trueskill.backends import cdf

Загружаем информацию про результаты игр, скачанную с http://dotascience.com/ и сразу сортируем по Match_ID.

In [2]:
matches_info = pd.read_csv('selected_team_matches.csv').sort(['match_id'])

Инициализируем параметры Trueskill используя параметры по умолчанию, кроме вероятность ничейного результата, у нас не бывает ничьих.

In [3]:
trueskill.setup(draw_probability=0)

trueskill.TrueSkill(mu=25.000, sigma=8.333, beta=4.167, tau=0.083, draw_probability=0.0%)

Создадим словарик, где будем хранить рейтинги. И обучим нашу систему. Для каждого матча мы определим победителя и пересчитаем рейтинги комманд. Если команда раньше не встречалась задаем ей рейтинг по умочанию. В нашей простейшей модели мы считаем, что каждая команда это один виртуальный игрок.

In [6]:
Rates = {}
for index, row in matches_info.iterrows():
    team1 = row['radiant']
    team2 = row['dire']
    if team1 not in Rates:
        Rates[team1] = trueskill.Rating()
    if team2 not in Rates:
        Rates[team2] = trueskill.Rating()
    if row['winner'] == 'RADIANT':
        Rates[team1], Rates[team2] = trueskill.rate_1vs1(Rates[team1], Rates[team2])
    else:
        Rates[team2], Rates[team1] = trueskill.rate_1vs1(Rates[team2], Rates[team1])
        

Теперь мы можем, например, насладится рейтингом команты Virtus.pro, которая скрывается под тагом 'VP 2' 

In [7]:
Rates['VP 2']

trueskill.Rating(mu=32.329, sigma=0.799)

Попробуем оценить вероятность победы Virtus.pro, например, в матче с Evil Genius.
К сожалению в библиотеке trueskill, нет возможности оценить вероятность, но мы же знаем из описания,
что рейтинги тут нормально распределенные. Так что сделаем функцию, которая оценит вероятность, 
что сила игры первой команды, будет больше чем у второй.

In [9]:
def win_probability(player_rating, opponent_rating):
    delta_mu = player_rating.mu - opponent_rating.mu
    denom = sqrt(2 * (BETA * BETA) + pow(player_rating.sigma, 2) + pow(opponent_rating.sigma, 2))
    return cdf(delta_mu / denom)

Ура, теперь мы можем оценить шансы.

In [10]:
win_probability(Rates['VP 2'], Rates['EG'])

0.48257973355975936

Система сказала, что шансы почти равные. Теперь полученную оценку можно загрузить на сайт в качетве прогноза. 
Как это сделать расказанно [тут](https://github.com/romovpa/dotascience-hackathon/blob/master/README.md).